In [6]:
# !pip3 install -U openai-whisper -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install noisereduce -i https://mirrors.aliyun.com/pypi/simple/ 

In [65]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from tqdm import tqdm
import torch
import librosa
import numpy as np
import glob
import os

import sys
sys.path.append("/home/shoinoue/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

import whisper
from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer
import pandas as pd
import scipy.stats as st
import math
from IPython.display import clear_output

sr = 16000

## Whisper

In [66]:
models = {
    "tiny": whisper.load_model("tiny.en"),
    "base": whisper.load_model("base.en"),
    "small": whisper.load_model("small.en"),
    "medium": whisper.load_model("medium.en"),
    "large": whisper.load_model("large"),
}

- ACed samples

In [67]:
files = glob.glob("./SPAT/transliteration/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = normalizer(text)

In [68]:
labels = {}
labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small"] = "only-gt_small"
labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating"] = "only-gt_large"
labels["VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating_small"] = "only-syn_small"
labels["VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating"] = "only-syn_large"
labels["VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating_small"] = "syn-gt_small"
labels["VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating"] = "syn-gt_large"
base_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"

In [5]:
wer_list = []
cer_list = []
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for size in models:
    wers = []
    cers = []
    for mn in labels:
        print(size, mn)
        files = glob.glob(base_dir + mn + "/100000/wav/*.wav")
        files.sort()

        predtexts = []
        gttexts = []
        for path in tqdm(files):
            basename = os.path.basename(path)[:-4]
            if not(basename in testsamples):
                continue
            predtext = normalizer(models[size].transcribe(path)["text"])
            predtexts += [predtext]
            gttexts += [groundtruth_dir[basename]]
        wer = jiwer.wer(gttexts, predtexts)
        cer = jiwer.cer(gttexts, predtexts)
        wers += [wer]
        cers += [cer]
    wer_list += [wers]
    cer_list += [cers]

tiny VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small


100%|██████████| 105/105 [00:27<00:00,  3.75it/s]


tiny VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating


100%|██████████| 105/105 [00:20<00:00,  5.20it/s]


tiny VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating_small


...

100%|██████████| 105/105 [01:25<00:00,  1.23it/s]


large VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating_small


100%|██████████| 100/100 [01:34<00:00,  1.06it/s]


large VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating


100%|██████████| 100/100 [01:50<00:00,  1.10s/it]


In [50]:
df = pd.DataFrame(np.array(wer_list), index=models.keys(), columns=labels.values())
df

NameError: name 'labels' is not defined

- Transcribe one sample

In [14]:
path = "example.wav"
# path = "voice_preview_Ken.wav"
# path = "voice_preview_Ena.wav"
for size in models:
    predtext = models[size].transcribe(path)["text"]
    print(predtext)

 Amid-dirt-kugali-legana-chai, or isip-ni-yatra-ke-in-dankirup-mejalana-chai.
 Hamed dar kugali lakanachai, or isya pni yatra ke indan ke rupe mejalanachai.
 Hamed Dardkul Galilaganachahi, Orise apni yatra ke Indian ke roop mejalana chahi.
 That's all for today. And As always, I'll see you guys in the next video.
 हमें दर्द को गले लगाना चाहिए और इसे अपनी यात्रा के इंधन के रूप में जलाना चाहिए।


# L2-arctic (all)

In [69]:
information = [
    ["ABA", "Arabic", "M"],
    ["SKA", "Arabic", "F"],
    ["YBAA", "Arabic", "M"],
    ["ZHAA", "Arabic", "F"],
    ["BWC", "Mandarin", "M"],
    ["LXC", "Mandarin", "F"],
    ["NCC", "Mandarin", "F"],
    ["TXHC", "Mandarin", "M"],
    ["ASI", "Hindi", "M"],
    ["RRBI", "Hindi", "M"],
    ["SVBI", "Hindi", "F"],
    ["TNI", "Hindi", "F"],
    ["HJK", "Korean", "F"],
    ["HKK", "Korean", "M"],
    ["YDCK", "Korean", "F"],
    ["YKWK", "Korean", "M"],
    ["EBVS", "Spanish", "M"],
    ["ERMS", "Spanish", "M"],
    ["MBMPS", "Spanish", "F"],
    ["NJS", "Spanish", "F"],
    ["HQTV", "Vietnamese", "M"],
    ["PNV", "Vietnamese", "F"],
    ["THV", "Vietnamese", "F"],
    ["TLV", "Vietnamese", "M"],
]
information = np.array(information)

In [70]:
files = glob.glob("./SPAT/transliteration/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = normalizer(text)

In [71]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
speakers = information[:,0]

In [72]:
load = True
save_path = "evaluation/wers/l2-arctic.npy"
if load:
    wer_list, cer_list = np.load(save_path)
else:
    wer_list = []
    cer_list = []
    testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
    for size in models:
        wers = []
        cers = []
        for spk in speakers:
            print(size, spk)
            spk_dir = base_dir + spk + "/wav/"
            predtexts = []
            gttexts = []
            for basename in tqdm(testsamples):
                path = spk_dir + basename + ".wav"
                if not(os.path.exists(path)):
                    continue
                predtext = normalizer(models[size].transcribe(path)["text"])
                predtexts += [predtext]
                gttexts += [groundtruth_dir[basename]]
            wer = jiwer.wer(gttexts, predtexts)
            cer = jiwer.cer(gttexts, predtexts)
            wers += [wer]
            cers += [cer]
        wer_list += [wers]
        cer_list += [cers]
    np.save(save_path, [wer_list, cer_list])

In [73]:
columns = []
accent_list = information[:, 1]
for i in range(len(speakers)):
    columns += [(accent_list[i], speakers[i])]
columns = pd.MultiIndex.from_tuples(columns)

In [93]:
df = pd.DataFrame(np.array(wer_list), index=models.keys(), columns=columns)
df = df.mean(level=0, axis=1)
df

,Arabic,Mandarin,Hindi,Korean,Spanish,Vietnamese
tiny,0.255182,0.274317,0.134426,0.160109,0.243915,0.355738
base,0.181688,0.220492,0.091803,0.128415,0.174764,0.302186
small,0.140225,0.157923,0.066940,0.093716,0.132419,0.242077
medium,0.104517,0.133060,0.057104,0.086612,0.110904,0.205738
large,0.412877,0.114481,0.046448,0.110929,0.087144,0.168306


- SPAccenT

In [56]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/"
speakers = [os.path.basename(os.path.dirname(a)) for a in glob.glob(base_dir+"*/")]
speakers.sort()
speakers.remove("Japanese")

In [57]:
load = True
save_path = "evaluation/wers/slt_spaccent.npy"
if load:
    wer_list, cer_list = np.load(save_path)
else:
    wer_list = []
    cer_list = []
    testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
    for size in models:
        wers = []
        cers = []
        for spk in speakers:
            spk_dir = base_dir + spk + "/wav/"
            predtexts = []
            gttexts = []
            for basename in tqdm(testsamples):
                path = spk_dir + basename + ".wav"
                if not(os.path.exists(path)):
                    continue
                predtext = normalizer(models[size].transcribe(path)["text"])
                predtexts += [predtext]
                gttexts += [groundtruth_dir[basename]]
            wer = jiwer.wer(gttexts, predtexts)
            cer = jiwer.cer(gttexts, predtexts)
            wers += [wer]
            cers += [cer]
        wer_list += [wers]
        cer_list += [cers]
    np.save(save_path, [wer_list, cer_list])

In [58]:
df = pd.DataFrame(np.array(wer_list), index=models.keys(), columns=speakers)
df

,English,Hindi,Korean
tiny,0.036066,0.202186,0.859016
base,0.026230,0.149727,0.748634
small,0.021858,0.113661,0.532240
medium,0.028415,0.110383,0.469945
large,0.017486,0.085246,0.498361


- L2-arctic

In [3]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
spk = "ASI"

In [4]:
files = glob.glob(base_dir + f"{spk}/transcript/*")
files.sort()
groundtruth_dir = {}
for file in files:
    f = open(file, "r")
    text = f.read()
    key = os.path.basename(file)[:-4]
    groundtruth_dir[key] = normalizer(text)

In [43]:
files = glob.glob("test/korean/arctic*.wav")
# files = glob.glob("test/gt/arctic*.wav")
# files = glob.glob("test/eng/arctic*.wav")
files.sort()

In [42]:
# Ground truth
for path in files:
    basename = os.path.basename(path)[:-4]
    predtext = normalizer(models[size].transcribe(path)["text"])
    wer = jiwer.wer(groundtruth_dir[basename], [predtext])
    # wer = jiwer.cer(groundtruth_dir[basename], [predtext])
    print(wer, predtext)

0.125 author of the danger trail philip still etc
0.25 not at this particular case tom apologized with more
0.0 for the 20th time that evening the 2 men shook hands
0.4 loads but i am glad to see your game feel
0.0 will we ever forget it
